In [1]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import re
import csv
import os

In [2]:
# Define paths
chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
user_data_dir = "/Users/federicopessina/ChromeProfile"  # Use a dedicated folder
remote_debugging_port = "9222"
factiva_url = "https://librarysearch.lse.ac.uk/discovery/fulldisplay?vid=44LSE_INST:44LSE_VU1&tab=Everything&docid=alma99129371110302021&context=L&search_scope=MyInstitution&lang=en"

# Launch Chrome with remote debugging
subprocess.Popen([
    chrome_path,
    f"--remote-debugging-port={remote_debugging_port}",
    f"--user-data-dir={user_data_dir}",
    factiva_url
])

# Give time for Chrome to open and for you to log in
print("Chrome launched. Log in manually, then press Enter to continue...")
input()


Chrome launched. Log in manually, then press Enter to continue...


[47060:4987576:0529/093213.714507:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions


DevTools listening on ws://127.0.0.1:9222/devtools/browser/d99ab401-e956-4c91-865c-55e6f43c17d5
[47060:4987519:0529/093217.985399:ERROR:chrome/browser/policy/cloud/fm_registration_token_uploader.cc:179] Client is missing for kUser scope
I0000 00:00:1748507538.008801 4987563 voice_transcription.cc:58] Registering VoiceTranscriptionCapability
Trying to load the allocator multiple times. This is *not* supported.
Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#-1 is a dynamic-sized tensor).
[47060:4987566:0529/093218.093768:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


''

In [3]:
options = Options()
options.add_experimental_option("debuggerAddress", "localhost:9222")

# Adjust path to your chromedriver
service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

time.sleep(3)

[47060:4987566:0529/093244.831730:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


In [14]:
print(driver.current_url)
print(len(driver.find_elements(By.ID, "dr")))  # dropdown
print(len(driver.find_elements(By.ID, "frd"))) # from-day field
print(len(driver.find_elements(By.ID, "laTab"))) # language tab
#driver.switch_to.window(driver.window_handles[-1])  # last tab
print(driver.current_url)

https://global-factiva-com.lse.idm.oclc.org/sb/default.aspx?lnep=hp
1
1
1
https://global-factiva-com.lse.idm.oclc.org/sb/default.aspx?lnep=hp


In [4]:
#### Change the date range ####
date_range = {'Start': '2020-01-01', 'End': '2020-01-07'}
start_day, start_month, start_year = date_range['Start'].split('-')[2], date_range['Start'].split('-')[1], date_range['Start'].split('-')[0]
end_day, end_month, end_year = date_range['End'].split('-')[2], date_range['End'].split('-')[1], date_range['End'].split('-')[0]

time.sleep(4)

# Select "Custom" date range
select = Select(driver.find_element(By.ID, "dr"))
select.select_by_visible_text("Enter date range...")
time.sleep(1.5)

# Fill in "From" date fields
driver.find_element(By.ID, 'frd').clear()
driver.find_element(By.ID, 'frd').send_keys(start_day)

driver.find_element(By.ID, 'frm').clear()
driver.find_element(By.ID, 'frm').send_keys(start_month)

driver.find_element(By.ID, 'fry').clear()
driver.find_element(By.ID, 'fry').send_keys(start_year)

# Fill in "To" date fields
driver.find_element(By.ID, 'tod').clear()
driver.find_element(By.ID, 'tod').send_keys(end_day)

driver.find_element(By.ID, 'tom').clear()
driver.find_element(By.ID, 'tom').send_keys(end_month)

driver.find_element(By.ID, 'toy').clear()
driver.find_element(By.ID, 'toy').send_keys(end_year)



In [16]:
#### Change language ####
time.sleep(4)
driver.find_element(By.ID, "laTab").click()
time.sleep(4)
english_link = driver.find_element(By.XPATH, "//a[text()='English' and @class='mnuItm']")
driver.execute_script("arguments[0].click();", english_link)

[21121:4365836:0528/132612.097212:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


In [17]:
#### Change region ####
time.sleep(4)
driver.find_element(By.XPATH, "//a[text()='Region' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(4)

# 2. Type "United Kingdom" in the search box
region_input = driver.find_element(By.ID, "reTxt")
region_input.clear()
region_input.send_keys("United Kingdom")
time.sleep(4)  # wait for dropdown to appear

# 3. Click button
driver.find_element(By.ID, "reLkp").click()
time.sleep(4)

# 4 find uk in list and click it
driver.find_element(By.XPATH, "//a[text()='United Kingdom' and contains(@class, 'mnuItm')]").click()
time.sleep(4)

In [ ]:
#### Industry ####
driver.find_element(By.XPATH, "//a[text()='Industry' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(4)

# 2. Type "Agriculture" in the input box
industry_input = driver.find_element(By.ID, "inTxt")
industry_input.clear()
industry_input.send_keys("Agriculture")
time.sleep(4)

# 3. Click the blue play/search icon
driver.find_element(By.ID, "inLkp").click()
time.sleep(4)

# 4. Click the "Agriculture" result
driver.find_element(By.XPATH, "//a[text()='Agriculture' and contains(@class, 'mnuItm')]").click()
time.sleep(4)


[47060:4987566:0529/093342.281980:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=137.0.7151.41)
Stacktrace:
0   chromedriver                        0x00000001048a03e0 cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x00000001048986a8 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x00000001043d5e30 cxxbridge1$string$len + 89744
3   chromedriver                        0x0000000104414cfc cxxbridge1$string$len + 347484
4   chromedriver                        0x00000001044129a0 cxxbridge1$string$len + 338432
5   chromedriver                        0x000000010445e044 cxxbridge1$string$len + 647332
6   chromedriver                        0x00000001044113f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000104864804 cxxbridge1$str$ptr + 2585200
8   chromedriver                        0x0000000104867ad4 cxxbridge1$str$ptr + 2598208
9   chromedriver                        0x0000000104845dd8 cxxbridge1$str$ptr + 2

In [10]:
# removing old industry when there's two of them
last_industry = 'Agriculture'
industry = 'Automotive'

driver.find_element(By.XPATH, "//a[text()='Industry' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(4)

# 2. Type the industry name in the input box
industry_input = driver.find_element(By.ID, "inTxt")
industry_input.clear()
industry_input.send_keys(industry)
time.sleep(4)

# 3. Click the blue play/search icon
driver.find_element(By.ID, "inLkp").click()
time.sleep(4)

# 4. Click the search result matching the industry name
driver.find_element(By.XPATH, f"//a[text()='{industry}' and contains(@class, 'mnuItm')]").click()
time.sleep(4)

old_label = driver.find_element(By.XPATH, f"//span[text()='{last_industry}' and contains(@class, 'filterText')]")
old_label.click()
time.sleep(1)

# Click the adjacent 'Remove' button that appears in dropdown
remove_button = driver.find_element(By.XPATH, "//span[text()='Remove']")
remove_button.click()
time.sleep(2)

In [ ]:
#### Subject ####

# Apply to all desired subjects
subjects_to_add = [
    "Commodity/Financial Market News",
    "Corporate/Industrial News",
    "Economic News"
]

def add_subject(subject_name):
    # Ensure the Subject tab is open before calling this
    subject_input = driver.find_element(By.ID, "nsTxt")
    subject_input.clear()
    subject_input.send_keys(subject_name)
    time.sleep(4)

    # Try clicking suggestion (strong) first
    try:
        driver.find_element(By.XPATH, f"//strong[text()='{subject_name}']/ancestor::td").click()
        print(f"Selected via suggestion: {subject_name}")
    except:
        # Fallback: click direct <a> tag
        driver.find_element(By.XPATH, f"//a[text()='{subject_name}' and contains(@class, 'mnuItm')]").click()
        print(f"Selected via <a>: {subject_name}")
    time.sleep(4)
    
    
# Step 1: Open the Subject tab once
driver.find_element(By.XPATH, "//a[text()='Subject' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(4)

for s in subjects_to_add:
    add_subject(s)
    
# Step 3 (optional): Close the Subject tab by clicking it again
driver.find_element(By.XPATH, "//a[text()='Subject' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(4)

Selected via suggestion: Commodity/Financial Market News


[21121:4365836:0528/132703.003762:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


Selected via suggestion: Corporate/Industrial News
Selected via suggestion: Economic News


In [20]:
# Add key search terms in search box at the top
driver.execute_script("""
    let editor = ace.edit(document.getElementsByClassName('ace_editor')[0]);
    editor.setValue('rst=tukn');
""")


In [21]:
# Remove some of the "more options"

driver.find_element(By.XPATH, "//a[contains(text(), 'More Options')]").click()
time.sleep(4)

# Check the box if not already checked
checkbox = driver.find_element(By.ID, "isteo_bool")
if not checkbox.is_selected():
    checkbox.click()
    time.sleep(4)

In [22]:
# Click the search button at the bottom
time.sleep(4)
driver.find_element(By.XPATH, '//*[@id="btnSearchBottom"]').click()
time.sleep(7)

In [ ]:
'''
wait = WebDriverWait(driver, 1)

result_count_cell = wait.until(EC.presence_of_element_located((
    By.XPATH,
    "//td[normalize-space()='Results Found']/following-sibling::td"
)))

# Extract and print the count
result_count = result_count_cell.text.strip()
print("Number of results found:", result_count)
'''

Number of results found: 16


In [41]:
# Extract rows from the summary table
summary_data = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='label']]")

summary_dict = {}
for row in summary_data:
    try:
        key = row.find_element(By.XPATH, "./td[1]").text.strip()
        value = row.find_element(By.XPATH, "./td[2]").text.strip()
        summary_dict[key] = value
    except:
        continue
        
print(summary_dict)

{'Text': 'rst=tukn', 'Date': '01/01/2020 to 07/01/2020', 'Source': 'All Sources', 'Author': 'All Authors', 'Company': 'All Companies', 'Subject': 'Commodity/Financial Market News Or Corporate/Industrial News Or Economic News', 'Industry': 'Agriculture', 'Region': 'United Kingdom', 'Language': 'English', 'Results Found': '16', 'Timestamp': '28 May 2025 14:27'}


In [42]:
try:
    dedup_value = driver.find_element(By.ID, "dedupSummary").text
    summary_dict["Deduplication Summary"] = dedup_value
except:
    summary_dict["Deduplication Summary"] = ""

In [43]:
industry_name = 'Agriculture'
filename = f"{date_range['Start']}_to_{date_range['End']}.csv"
folder = f"factiva_results/{industry_name}"
os.makedirs(folder, exist_ok=True)
# Save as one-row CSV
csv_path = os.path.join(folder, filename)
with open(csv_path, "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=summary_dict.keys())
    writer.writeheader()
    writer.writerow(summary_dict)

print(f"Saved to {csv_path}")

Saved to factiva_results/Agriculture/2020-01-01_to_2020-01-07.csv
